In [14]:
import pandas as pd
import numpy as np
import tmdbsimple as tmdb
import os, time,json
from tqdm.notebook import tqdm_notebook

FOLDER = "data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [9]:
with open('/Users/ryned/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)

login.keys()

tmdb.API_KEY = login['api-key']

In [10]:
YEARS_TO_GET = [year for year in range(2010,2020)]
errors = [ ]

basics = pd.read_csv(r'data\title_basics.csv.gz')

In [11]:
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    info = movie.info()
    releases = movie.releases()

    for c in releases['countries']:
        if c['iso_3166_1' ] =='US':
            info['certification'] = c['certification']

    return info

In [12]:
def write_json(new_data, filename):

    with open(filename,'r+') as file:
        file_data = json.load(file)
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        file.seek(0)

        json.dump(file_data, file)


In [15]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    df = basics.loc[ basics['startYear']==YEAR].copy()

    movie_ids = df['tconst'].copy()
    movie_ids

    previous_df = pd.read_json(JSON_FILE)
    previous_df

    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    for movie_id in tqdm_notebook(movie_ids_to_get,
                        desc=f'Movies from {YEAR}',
                        position=1,
                        leave=True):
        try:
            temp = get_movie_with_rating(movie_id)  
            write_json(temp,JSON_FILE)
            time.sleep(0.02)
    
        except Exception as e:
            errors.append([movie_id, e])

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/10 [00:00<?, ?it/s]

Movies from 2010:   0%|          | 0/3848 [00:00<?, ?it/s]

Movies from 2011:   0%|          | 0/4224 [00:00<?, ?it/s]

Movies from 2012:   0%|          | 0/4510 [00:00<?, ?it/s]

Movies from 2013:   0%|          | 0/4702 [00:00<?, ?it/s]

...

Movies from 2015:   0%|          | 0/5054 [00:00<?, ?it/s]

Movies from 2016:   0%|          | 0/5249 [00:00<?, ?it/s]

Movies from 2017:   0%|          | 0/5642 [00:00<?, ?it/s]

Movies from 2018:   0%|          | 0/5771 [00:00<?, ?it/s]

Movies from 2019:   0%|          | 0/5874 [00:00<?, ?it/s]